In [ ]:
import pandas as pd
import numpy as np
import requests
from io import StringIO
import ast
import aiohttp
import asyncio
from itertools import chain
import datetime
import calendar
from bs4 import BeautifulSoup
import sqlite3

pd.set_option('display.max_columns', False)
pd.options.mode.chained_assignment = None

In [ ]:
# Прикол на будущее, хочу сохранять историю торгов

connection = sqlite3.connect('bonds_database.db')
cur = connection.cursor()

max_tradedate = cur.execute('SELECT MAX(TRADEDATE) FROM bonds_table') \
    .fetchone()[0]
max_date = datetime.datetime.strptime(max_tradedate, '%Y-%m-%d').date()

In [ ]:
todays_date = datetime.datetime.now().date()
days_from_last_parse = (todays_date-max_date).days

first_year_date = datetime.date(
    datetime.datetime.now().year, 1, 1
)
last_year_date = datetime.date(
    datetime.datetime.now().year, 12, 31
)
days_in_year = (last_year_date - first_year_date).days

In [ ]:
cols = requests.get("https://iss.moex.com/iss/history/engines/stock/markets/bonds/securities.json?numtrades=1&limit=1").json().get('history').get('columns')

In [ ]:
lst_of_urls = []
for i in range(1, days_from_last_parse):
    parsing_day = max_date + datetime.timedelta(i)
    if parsing_day.weekday() != 5 and parsing_day.weekday() != 6:
        for start_num in range(0, 50001, 100):
            url_of_100bonds = f"https://iss.moex.com/iss/history/engines/stock/markets/bonds/securities.json?numtrades=1&start={start_num}&date={parsing_day.strftime('%Y-%m-%d')}"
            lst_of_urls.append(url_of_100bonds)
lst_of_urls

In [ ]:
async def fetch(url, session):
    async with session.get(url) as response:
        resp = await response.json()
        history = resp.get('history')
        data = history.get('data')
        if len(data) != 0 or data != None:
            return data


async def fetch_all(urls):
    async with aiohttp.ClientSession() as session:
        tasks = [fetch(url, session) for url in urls]
        results = await asyncio.gather(*tasks, return_exceptions=False)
        return results

third_dimens_lst_records = await fetch_all(lst_of_urls)
records = list(chain(*third_dimens_lst_records))
df = pd.DataFrame.from_records(data=records, columns=cols)

df.to_sql('raw_bonds_table', connection, if_exists='append', index=False)

In [ ]:
# Обработка

df = df[df.OFFERDATE.isna()]
df = df[~df.MATDATE.isna()] 
df = df[~df.DURATION.isna()]
df = df[df.BUYBACKDATE.isna()]
df = df[df.COUPONPERCENT != 0]
df = df[df.COUPONVALUE != 0]
df = df[df.ACCINT != 0]

df_clean = df[['BOARDID', 'TRADEDATE', 'SHORTNAME', 'SECID', 'NUMTRADES', 'CLOSE', 'ACCINT', 'MATDATE', 'DURATION', 'COUPONPERCENT', 'COUPONVALUE', 'FACEVALUE', 'FACEUNIT']]

In [ ]:
df_clean.MATDATE = pd.to_datetime(df_clean.MATDATE).dt.date
df_clean['right_duration'] = (df_clean.MATDATE - todays_date).dt.days

df_clean['coupon_times_year'] = round(df_clean.COUPONPERCENT / (df_clean.COUPONVALUE / df_clean.FACEVALUE * 100))

df_clean['bid'] = df_clean.FACEVALUE * df_clean.CLOSE / 100

df_clean['accint_percentage'] = df_clean.ACCINT /  df_clean.COUPONVALUE

df_clean['coupons_interval'] = round(days_in_year / df_clean.coupon_times_year).astype('timedelta64[D]')
df_clean = df_clean[~df_clean.coupons_interval.isna()]

df_clean['coupon_all_times'] = (df_clean.right_duration / df_clean.coupons_interval.dt.days).apply(np.floor) + 1

df_clean['next_coupon_date'] = df_clean.MATDATE - ((df_clean.coupon_all_times - 1) * df_clean.coupons_interval)

df_clean = df_clean.replace(np.inf, np.nan)

df_clean['simple_revenue'] = (df_clean.coupon_all_times * df_clean.COUPONVALUE - df_clean.ACCINT) + (df_clean.FACEVALUE - df_clean.bid)
df_clean['simple_percent_revenue'] = df_clean.simple_revenue / df_clean.bid * 100 / df_clean.right_duration * days_in_year

df_clean['profit'] = np.where(df_clean.right_duration < days_in_year * 3,
    df_clean.simple_revenue - (df_clean.bid * 0.0003) - (df_clean.simple_revenue * 0.13),
    df_clean.simple_revenue - (df_clean.bid * 0.0003))
df_clean['percent_profit'] = df_clean.profit / df_clean.bid * 100 / df_clean.right_duration * days_in_year

target_secids = df_clean.SECID.unique()

In [ ]:
async def fetch_smart_lab(url, session):
    async with session.get(url) as response:
        resp = await response.text()
        soup = BeautifulSoup(resp, 'lxml')
        parsed_data = soup.find_all('div', class_="quotes-simple-table__item")
        parsed_data = list(
            map(lambda x: x.text, parsed_data
        ))

        translation_dict = {
        '\n': '',
        '\t': ''
        }
        transfromed_data = list(
            map(
                lambda x: x.translate(x.maketrans(translation_dict)),
                                      parsed_data)
            )
        return transfromed_data

async def fetch_all_smart_lab(urls):
    async with aiohttp.ClientSession() as session:
        tasks = [fetch_smart_lab(url, session) for url in urls]
        results = await asyncio.gather(*tasks, return_exceptions=False)
        return results

In [ ]:
base_url = 'https://smart-lab.ru/q/bonds/'
smart_lab_url_lst = []
for sic_id in target_secids:
    smart_lab_url_lst.append(base_url+sic_id)

lst_of_parser_info = await fetch_all_smart_lab(smart_lab_url_lst)

In [ ]:
additional_info = []
parsed_data_dict_keys = ['Только для квалов?', 'Кредитный рейтинг', 'Сектор', 'ISIN']
for rec in lst_of_parser_info:
    rec_lst = []
    for target_value in parsed_data_dict_keys:
        try:
            rec_lst.append(rec[rec.index(target_value)+1])
        except:
            rec_lst.append(np.nan)
    additional_info.append(rec_lst)

additional_info_df = pd.DataFrame().from_records(additional_info, columns=parsed_data_dict_keys) \
    .rename(columns={'ISIN': 'SECID'}) \
    .drop_duplicates()
bonds_df = df_clean.merge(additional_info_df, on='SECID', how='left')

In [ ]:
# Добавление в бд
bonds_df.to_sql('bonds_table', connection, if_exists='append', index=False)

cur.close()
connection.close()

In [ ]:
filtered_df = bonds_df[
    (bonds_df['Только для квалов?'] != 'Да') &
    (bonds_df.percent_profit > 0) &
    (bonds_df.accint_percentage <= 0.4)
].sort_values(['percent_profit', 'Кредитный рейтинг'], ascending=[False,True])

short_dur = filtered_df[filtered_df.right_duration < days_in_year]
mid_dur = filtered_df[(days_in_year < filtered_df.right_duration) &
                      (days_in_year*2 > filtered_df.right_duration)]
long_dur = filtered_df[filtered_df.right_duration > days_in_year*2]

In [ ]:
# Описание колонок

# columns_description = requests.get("https://iss.moex.com/iss/history/engines/stock/markets/bonds/securities/columns.json").json().get('history').get('data')
# for i in columns_description:
#     print(f'{i[1]} - {i[2]} - {i[3]}')